In [27]:
import os

import xarray as xr
import pandas as pd
import numpy as np

import holoviews as hv
# from holoviews.operation.datashader import datashade, rasterize
# from holoviews.operation import decimate
# hv.extension('bokeh')
import hvplot.xarray
# import hvplot.pandas

from hvplot import hvPlot

import panel as pn


In [29]:
SAMPLE_COUNT = 100_000

In [30]:
samples = np.random.randint(-100, 100, (3,2,SAMPLE_COUNT), dtype=np.int32)
samples

array([[[-89,  90,  27, ...,  71,  23, -70],
        [-75, -15,  23, ..., -21, -56, -35]],

       [[ 76, -92,  66, ...,  92,  30, -76],
        [ 51, -72,  25, ...,  -6,  87,  19]],

       [[-17,  46, -53, ...,  12, -17, -16],
        [-99, -40, -92, ..., -14,  28, -35]]], dtype=int32)

In [31]:
waveforms = xr.DataArray(
    samples, 
    dims=('waveform', 'channel', 'sample'),
    coords={
        'channel': ['A', 'B'],
        'sample': range(SAMPLE_COUNT),
    },
    name='waveforms',
    attrs={
        'sample_rate': 1e9
    })
waveforms

<xarray.DataArray 'waveforms' (waveform: 3, channel: 2, sample: 100000)>
array([[[-89,  90,  27, ...,  71,  23, -70],
        [-75, -15,  23, ..., -21, -56, -35]],

       [[ 76, -92,  66, ...,  92,  30, -76],
        [ 51, -72,  25, ...,  -6,  87,  19]],

       [[-17,  46, -53, ...,  12, -17, -16],
        [-99, -40, -92, ..., -14,  28, -35]]], dtype=int32)
Coordinates:
  * channel  (channel) <U1 'A' 'B'
  * sample   (sample) int64 0 1 2 3 4 5 ... 99994 99995 99996 99997 99998 99999
Dimensions without coordinates: waveform
Attributes:
    sample_rate:  1000000000.0

In [32]:
metadata = pd.DataFrame(
    data=(
        ('wf 0', 0),
        ('wf 1', 1),
        ('wf 2', 1),
    ),
    columns=['title','group']
)
metadata.index.name = 'measurement'
metadata

,title,group
measurement,,
0,wf 0,0
1,wf 1,1
2,wf 2,1


In [33]:
ds = xr.Dataset(
    data_vars={
        'waveforms': waveforms,
        'metadata': metadata,
    }
)
ds

<xarray.Dataset>
Dimensions:      (channel: 2, sample: 100000, waveform: 3, measurement: 3,
                  dim_1: 2)
Coordinates:
  * channel      (channel) <U1 'A' 'B'
  * sample       (sample) int64 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
  * measurement  (measurement) int64 0 1 2
  * dim_1        (dim_1) object 'title' 'group'
Dimensions without coordinates: waveform
Data variables:
    waveforms    (waveform, channel, sample) int32 -89 90 27 -41 ... -14 28 -35
    metadata     (measurement, dim_1) object 'wf 0' 0 'wf 1' 1 'wf 2' 1

In [34]:
wf = ds.waveforms.sel(channel='A', waveform=0)
plt = hvPlot(wf)
plt.line(rasterize=True).opts(
    width=1200,
    height=400,
    toolbar='below',
    tools=['xpan', 'ypan', 
           'xbox_zoom', 'ybox_zoom',
           'xzoom_in', 'xzoom_out', 'yzoom_in', 'yzoom_out',
           'xwheel_zoom', 'ywheel_zoom',
           'xwheel_pan', 'ywheel_pan',
           'undo', 'redo'],
    # active_tools=['xbox_zoom', 'xwheel_pan']
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'b47e0a2e-6204-4b49-98f0-83b54a361cb2': {'version…

Task exception was never retrieved
future: <Task finished name='Task-33' coro=<Callback.process_on_change() done, defined at /home/nils/projects/wf-python/.venv/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py:322> exception=UnsetValueError("figure(id='p2873', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/home/nils/projects/wf-python/.venv/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 340, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/home/nils/projects/wf-python/.venv/lib/python3.10/site-packages/holoviews/plotting/bokeh/callbacks.py", line 248, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/home/nils/projects/wf-python/.venv/lib/python3.10/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descriptors.UnsetValueError: figu

In [9]:
# wf = ds.measurements.sel(channel='A', measurement=0)
# curve = hv.Curve(wf.data)
# rasterize(curve).opts(
#     width=1200,
#     height=400,
#     toolbar='below',
#     tools=['xpan', 'ypan', 
#            'xbox_zoom', 'ybox_zoom',
#            'xzoom_in', 'xzoom_out', 'yzoom_in', 'yzoom_out',
#            'xwheel_zoom', 'ywheel_zoom',
#            'xwheel_pan', 'ywheel_pan',
#            'undo', 'redo'],
#     # active_tools=['xbox_zoom', 'xwheel_pan']
# )

In [26]:
wf_slider = pn.widgets.DiscreteSlider(name='waveform', options=list(ds.waveform.values))
wf_select = pn.widgets.Select(name='channel', options=list(ds.channel.values))

wfi = ds.waveforms.interactive()
wfi.sel(channel=wf_select, waveform=wf_slider).hvplot(
    width=1200,
    height=400,
    # toolbar='below',
    tools=['xpan', 'ypan', 
           'xbox_zoom', 'ybox_zoom',
           'xzoom_in', 'xzoom_out', 'yzoom_in', 'yzoom_out',
           'xwheel_zoom', 'ywheel_zoom',
           'xwheel_pan', 'ywheel_pan',
           'undo', 'redo'],
)


BokehModel(combine_events=True, render_bundle={'docs_json': {'c8cf8c0e-f3ff-4528-8e70-f01513231028': {'version…